# Primera entrega

## Limpieza de datos
En esta entrega se hará una recolección y limpieza de datos solamente. No habrá modelado ni análisis. 
<br>Solamente velar por la integridad, coherencia y cohesión del conjunto de datos. Se realizará una transformación de formato inicial de archivos html disfrazados de .xls a archivos .csv para mayor facilidad de manejo.

In [ ]:
# !pip install -r requirements.txt

In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

In [3]:
def transform_html_to_csv(input_path: str, output_path: str=None, tables_to_convert: list[int] = [9]) -> None:
    '''
    Turns .html files into .csv files

    Params:
        input_path: where the .html files are stored
        output_path: where the .csv will be stored
    Returns:
        out: None
    '''
    if output_path is None:
        output_path = input_path 
    else:
        os.makedirs(output_path, exist_ok=True)

    for filename in tqdm(os.listdir(input_path), desc='Converting Files'):
        if filename.endswith('.xls'):
            html_path:str = os.path.join(input_path, filename)
            base_name:str = os.path.splitext(filename)[0]
            # try to read the excel and copy it into a .csv file
            try: 
                tables:list[pd.DataFrame] = pd.read_html(html_path)
                # if no tables, skip
                if not tables:
                    print(f"No tables found in '{filename}'")
                    continue

                # convert each of the tables selected to .csv
                for i, df in enumerate(tables):
                    if i in tables_to_convert:
                        csv_filename = f"{base_name}_table{i}.csv"
                        csv_path = os.path.join(output_path, csv_filename)
                        df.to_csv(csv_path, index=False)
                        print(f"Converted '{filename}' to '{csv_filename}'") # print all of the transformations of the .xls to its respective .csv files
                    
            except Exception as e:
                print(f"Error processing file '{filename}' : {e}")

def remove_files(path: str, extension: str) -> None:
    '''
    Removes all files that have a certain extension
    Input:
        path: folder path where .xls to be removed are stored
    '''
    for filename in os.listdir(path):
        if filename.endswith(extension):
            try:
                xls_path = os.path.join(path, filename)
                os.remove(xls_path)
            except Exception as e:
                print(f"Error removing file '{filename}' : {e}")


In [ ]:
# start by turning .xls files to .csv files
transform_html_to_csv(input_path="../Dataset_raw", output_path="../Dataset_cleaned", tables_to_convert=[9])

In [4]:
# remove .xls files if necessary
remove_files(path="../Dataset_raw", extension='.xls')